In [37]:
# set up
import os
import pandas as pd
import numpy as np

# Define folder paths relative to this notebook (under stage06_data-preprocessing)
raw_dir = './data/raw'
processed_dir = './data/processed'
src_dir = './src'

# Create folders if they don't exist
os.makedirs(raw_dir, exist_ok=True)
os.makedirs(processed_dir, exist_ok=True)
os.makedirs(src_dir, exist_ok=True)

# Define the sample data
data = {
    'age': [34, 45, 29, 50, 38, np.nan, 41],
    'income': [55000, np.nan, 42000, 58000, np.nan, np.nan, 49000],
    'score': [0.82, 0.91, np.nan, 0.76, 0.88, 0.65, 0.79],
    'zipcode': ['90210', '10001', '60614', '94103', '73301', '12345', '94105'],
    'city': ['Beverly', 'New York', 'Chicago', 'SF', 'Austin', 'Unknown', 'San Francisco'],
    'extra_data': [np.nan, 42, np.nan, np.nan, np.nan, 5, np.nan]
}

# Create DataFrame
df = pd.DataFrame(data)

# Save to CSV in raw data folder
csv_path = os.path.join(raw_dir, 'sample_data.csv')
if not os.path.exists(csv_path):
    df.to_csv(csv_path, index=False)
    print(f'Sample dataset created and saved to {csv_path}')
else:
    print(f'File already exists at {csv_path}. Skipping CSV creation to avoid overwrite.')


Sample dataset created and saved to ./data/raw/sample_data.csv


In [44]:
df = pd.read_csv('./data/raw/sample_data.csv')
df.head()

,age,income,score,zipcode,city,extra_data
0,34.0,55000.0,0.82,90210,Beverly,NaN
1,45.0,NaN,0.91,10001,New York,42.0
2,29.0,42000.0,NaN,60614,Chicago,NaN
3,50.0,58000.0,0.76,94103,SF,NaN
4,38.0,NaN,0.88,73301,Austin,NaN


In [23]:
# Basic info and missing counts
df.info()
df.isna().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   age         6 non-null      float64
 1   income      4 non-null      float64
 2   score       6 non-null      float64
 3   zipcode     7 non-null      int64  
 4   city        7 non-null      object 
 5   extra_data  2 non-null      float64
dtypes: float64(4), int64(1), object(1)
memory usage: 468.0+ bytes


age           1
income        3
score         1
zipcode       0
city          0
extra_data    5
dtype: int64

In [64]:
import sys
sys.path.append("./src")

from cleaning import (
    fill_missing_median,
    drop_missing,
    normalize_data,
    correct_column_types,
)

# choose numeric columns explicitly (avoids touching zipcode/city)
numeric_cols = ['age', 'income', 'score', 'extra_data']

# A) correct obvious types (no-ops here but keeps parity with example)
df1 = correct_column_types(df)

# B) fill missing numeric with median
df2 = fill_missing_median(df1, columns=numeric_cols)

# C) drop rows that still have too many missing values in key columns (example: keep ≥80% non-null)
df3 = drop_missing(df2, columns=numeric_cols, threshold=0.8)

# D) normalize numeric columns (minmax like the example)
df4 = normalize_data(df3, columns=numeric_cols, method='minmax')

df4.head()


,age,income,score,zipcode,city,extra_data
0,0.238095,0.8125,0.653846,90210,Beverly,0.5
1,0.761905,0.6250,1.000000,10001,New York,1.0
2,0.000000,0.0000,0.596154,60614,Chicago,0.5
3,1.000000,1.0000,0.423077,94103,SF,0.5
4,0.428571,0.6250,0.884615,73301,Austin,0.5


In [71]:
df2.to_csv('./data/processed/sample_data_cleaned.csv', index=False)
df2.head()  # after fill_missing_median, before normalization

,age,income,score,zipcode,city,extra_data
0,34.0,55000.0,0.820,90210,Beverly,23.5
1,45.0,52000.0,0.910,10001,New York,42.0
2,29.0,42000.0,0.805,60614,Chicago,23.5
3,50.0,58000.0,0.760,94103,SF,23.5
4,38.0,52000.0,0.880,73301,Austin,23.5
